In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle
from reportlab.lib.units import inch
from langchain.schema import HumanMessage
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import DataFrameLoader
from langchain_groq import ChatGroq
import json

# Servidor local do Ollama
#ollama_server_url = "http://localhost:11434"
model_local = llm = ChatGroq(
    model="llama3-groq-70b-8192-tool-use-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=""
)

# Template

rag_template = """
    Você é um assistente cuja função é extrair informações do e-mail {input}, você deve retornar as seguintes informações:
        1. Equipamentos de laboratório necessários

    e retornar um json com as seruintes informações encontradas para cada equipamento encontrado no formato capitalize:

        1. Nome
        2. Quantidade

"""

# Criação do primeiro prompt
prompt = ChatPromptTemplate.from_template(rag_template)

# Texto de exemplo
texto = """

"""

# Gera o prompt para o primeiro agente
final_prompt = prompt.format(input=texto)

# Converte para HumanMessage
message = [HumanMessage(content=final_prompt)]

# Executa o modelo (primeiro agente)
response = model_local.invoke(message)

data = json.loads(response.content)

def rodape(canvas, doc):
    # Salva o estado do canvas
    canvas.saveState()
    # Define a fonte e o tamanho
    canvas.setFont('Helvetica', 10)
    # Posiciona o rodapé (texto final) na parte inferior da página
    largura_pagina, altura_pagina = A4
    texto_rodape = "Av. Unisinos, 950 Caixa postal 275 CEP 93022-000 São Leopoldo Rs Brasil"
    canvas.drawString(100, 40, texto_rodape)
    # Restaura o estado do canvas
    canvas.restoreState()

def criar_pdf_alocacao(nome_arquivo, dados_equipamentos):
    # Cria o documento PDF
    pdf = SimpleDocTemplate(nome_arquivo, pagesize=A4)

    # Lista de elementos que compõem o documento
    elementos = []

    # Estilo básico de texto
    estilos = getSampleStyleSheet()

    # Adicionando um título
    titulo = Paragraph("TERMO DE COMPROMISSO PARA RETIRADA DE EQUIPAMENTOS", estilos['Title'])
    elementos.append(titulo)

    # Espaçamento
    elementos.append(Spacer(1, 20))

    # Parágrafo introdutório
    introducao = Paragraph("Eu _______________________________________ comprometo-me em entregar os equipamentos pertencentes a Universidade do Vale do Rio dos SInos - UNISINOS que estão sob responsabilidade do Laboratório de Sensoriamento Remoto e Cartografia Didital - LASERCA, devolvendo-os <b>íntegros e limpos</b> até o dia ____ de __________ de 20____.", estilos['BodyText'])
    elementos.append(introducao)

    # Espaçamento
    elementos.append(Spacer(1, 12))

    # Tabela de dados de alocação
    dados = [
        ['Equipamento', 'Quantidade', 'Patrimonio', 'Status']
    ]

    # Adiciona os dados dos equipamentos na tabela
    for equipamento in dados_equipamentos:
        dados.append([equipamento['Nome'].capitalize(), equipamento['Quantidade']])

    tabela = Table(dados, colWidths=[2.5 * inch, 1.5 * inch, 1.5 * inch, 2 * inch])

    # Estilo da tabela
    estilo_tabela = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.white),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ])
    tabela.setStyle(estilo_tabela)

    # Adiciona a tabela ao documento
    elementos.append(tabela)

    # Espaçamento
    elementos.append(Spacer(1, 12))

    # Rodapé (com campos para assinatura)
    destino = Paragraph("Destino do Campo: _______________________________________________________________ <br/><br/> Assinaturo do Responsavel: ________________________________________________________ <br/><br/> Retirada: ____ de __________ de 20____ Ass. Funcionario _______________________________<br/><br/> Devolução: ____ de __________ de 20____ Ass. Funcionario _____________________________", estilos['Normal'])
    elementos.append(destino)

    # Gera o PDF, adicionando o rodapé em cada página
    pdf.build(elementos, onFirstPage=rodape, onLaterPages=rodape)


# Gerando o PDF com os dados fornecidos
criar_pdf_alocacao("alocacao_equipamentos.pdf", data['Equipamentos de laboratório necessários'])